In [4]:
import torch
import numpy as np
from matplotlib import pyplot as plt 
import torch.nn as nn 
import torch.nn.functional as F 
from torchvision import transforms, datasets

In [3]:
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else :
  DEVICE = torch.device('cpu')
print(torch.cuda.get_device_name())

NVIDIA GeForce GTX 1650


In [5]:
print('using Pytorch version:', torch.__version__, 'Device: ', DEVICE)

using Pytorch version: 1.12.0 Device:  cuda


In [6]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
train_dataset = datasets.MNIST(root='../data/MNIST',
                               train = True,
                               download=True,
                               transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root= '../data/MNIST',
train=False,
download=True,
transform = transfroms.ToTensor())